In [1]:
import pandas as pd
import geopandas as gpd
import requests
import time
from shapely.geometry import Point
from geopandas.tools import sjoin

In [2]:
# Import of the Polygons
gdf_combined = gpd.read_file("C:/Users/edoar/combined_quartieri.geojson")

# API di YELP

In [6]:
neighborhoods = [f"{name}, Milan" for name in gdf_combined["Neighborhood"].tolist()]
neighborhoods

['Parco delle Abbazie, Milan',
 'Adriano, Milan',
 'Affori, Milan',
 'Baggio, Milan',
 'Bande Nere, Milan',
 'Barona, Milan',
 'Bicocca, Milan',
 'Bovisasca, Milan',
 'Bovisa, Milan',
 'Brera, Milan',
 'Bruzzano, Milan',
 'Buenos Aires - Venezia, Milan',
 'Cantalupa, Milan',
 'Cascina Triulza - Expo, Milan',
 'Centrale, Milan',
 'Chiaravalle, Milan',
 'Città Studi, Milan',
 'Comasina, Milan',
 'Corsica, Milan',
 'De Angeli - Monte Rosa, Milan',
 'Dergano, Milan',
 'Duomo, Milan',
 'Ex Om - Morivione, Milan',
 'Farini, Milan',
 'GARIBALDI REPUBBLICA, Milan',
 'Gallaratese, Milan',
 'Ghisolfa, Milan',
 'Giambellino, Milan',
 'Giardini Porta Venezia, Milan',
 'Gratosoglio - Ticinello, Milan',
 'Greco, Milan',
 'Guastalla, Milan',
 'Isola, Milan',
 'Lambrate, Milan',
 'Lodi - Corvetto, Milan',
 'Lorenteggio, Milan',
 'Loreto, Milan',
 'Maciachini - Maggiolina, Milan',
 'Magenta - San Vittore, Milan',
 'Maggiore - Musocco, Milan',
 'Mecenate, Milan',
 'Muggiano, Milan',
 'Navigli, Milan',
 

In [8]:
# Replace with your Yelp API key
API_KEY = "6CbFJGYPopltfBbkb00hIFGRI4XBrAsccPevTJ53ol4YIuJrF48kEylTmKvEl4-TEP8p0pEc3ydzUsatpgGI5aqSHCsysi5-yWyeJD3V-6al5x6_AcFkw23LehKEZ3Yx"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

# Constants
businesses_per_request = 48  # Maximum allowed by Yelp per request

def make_request(url, params=None):
    """
    Makes a request to the Yelp API.
    """
    response = requests.get(url, headers=HEADERS, params=params)
    response.raise_for_status()
    return response

def search_businesses(location, term="nightlife", limit=businesses_per_request, offset=0):
    """
    Searches for nightlife businesses in a given location with pagination.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "location": location,  # Neighborhood or address
        "term": term,  # Term like "nightlife", "bar", "club"
        "limit": limit,
        "offset": offset
    }
    response = make_request(url, params=params)
    return response.json().get("businesses", [])

# Initialize data storage
data = []

try:
    for neighborhood in neighborhoods:  # Loop through each neighborhood
        offset = 0
        while True:  # Keep fetching data until no more results
            print(f"Fetching nightlife in {neighborhood} with offset: {offset}...")

            try:
                # Fetch businesses using the current offset and location
                businesses = search_businesses(location=neighborhood, term="nightlife", limit=businesses_per_request, offset=offset)

                if not businesses:
                    # No more businesses to fetch
                    print(f"No more nightlife venues returned for {neighborhood}.")
                    break

                for biz in businesses:
                    name = biz.get("name", "N/A")
                    location_info = biz.get("location", {})
                    address = location_info.get("address1", "N/A")
                    categories = biz.get("categories", [])
                    category_list = [cat.get("title", "") for cat in categories if cat.get("title")]
                    category_str = ", ".join(category_list) if category_list else "N/A"
                    rating = biz.get("rating", "N/A")
                    review_count = biz.get("review_count", "N/A")

                    # Extract coordinates
                    coordinates = biz.get("coordinates", {})
                    latitude = coordinates.get("latitude", None)
                    longitude = coordinates.get("longitude", None)

                    # Append to data
                    data.append({
                        "Fetch Location": neighborhood,
                        "Venue Name": name,
                        "Venue Address": address,
                        "Categories": category_str,
                        "Average Star Rating": rating,
                        "Review Count": review_count,
                        "Latitude": latitude,
                        "Longitude": longitude
                    })

                # Increment offset for the next batch
                offset += len(businesses)

                # Optional: Sleep to respect API rate limits
                time.sleep(0.5)

                # Break if the offset exceeds Yelp's maximum results per query
                if offset >= 240:  # Maximum 240 results per query
                    print(f"Reached maximum results for {neighborhood}.")
                    break

            except requests.HTTPError as he:
                # Log the error and skip this neighborhood
                print(f"HTTP error occurred for {neighborhood}: {he}")
                break

    # Convert the collected data into a DataFrame
    df = pd.DataFrame(data)

    # Convert DataFrame to GeoDataFrame
    df["geometry"] = df.apply(
        lambda row: Point(row["Longitude"], row["Latitude"]) if row["Longitude"] and row["Latitude"] else None,
        axis=1
    )
    Nightlife = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    # Print summary
    print(Nightlife.head())
    print(f"Total nightlife venues collected: {len(data)}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

Fetching nightlife in Parco delle Abbazie, Milan with offset: 0...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 48...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 96...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 144...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 192...
Reached maximum results for Parco delle Abbazie, Milan.
Fetching nightlife in Adriano, Milan with offset: 0...
HTTP error occurred for Adriano, Milan: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=Adriano%2C+Milan&term=nightlife&limit=48&offset=0
Fetching nightlife in Affori, Milan with offset: 0...
Fetching nightlife in Affori, Milan with offset: 48...
Fetching nightlife in Affori, Milan with offset: 96...
Fetching nightlife in Affori, Milan with offset: 144...
Fetching nightlife in Affori, Milan with offset: 192...
Reached maximum results for Affori, Milan.
Fetching nightlife in Baggio, Milan with offset: 

In [10]:
Nightlife

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry
0,"Parco delle Abbazie, Milan",H2O Live Club,Via Mezzomerico 18,Dance Clubs,0.0,0,45.626890,8.625470,POINT (8.62547 45.62689)
1,"Parco delle Abbazie, Milan",049,Piazza Martiri della Libertà 3B,"Lounges, Wine Bars, Cocktail Bars",4.7,3,45.445177,8.616799,POINT (8.6168 45.44518)
2,"Parco delle Abbazie, Milan",Twiggy Club,Via de Cristoforis 5,"Music Venues, Pubs, Cafes",5.0,1,45.821506,8.831431,POINT (8.83143 45.82151)
3,"Parco delle Abbazie, Milan",Circle,Via Stendhal 34,"Lounges, Dance Clubs",3.4,7,45.453430,9.159440,POINT (9.15944 45.45343)
4,"Parco delle Abbazie, Milan",Strix Bar,Via Olgiate Olona 1,Cocktail Bars,4.0,2,45.620991,8.865000,POINT (8.865 45.62099)
...,...,...,...,...,...,...,...,...,...
17827,"Forze Armate, Milan",Salumeria dello Sport,PIazza Napoli 30,"Sports Bars, Wine Bars, Cocktail Bars",0.0,0,45.453099,9.152149,POINT (9.15215 45.4531)
17828,"Forze Armate, Milan",Square Bar,Via Michelangelo Buonarroti 5,Bars,3.4,10,45.467357,9.155144,POINT (9.15514 45.46736)
17829,"Forze Armate, Milan",I Beerbanti,Via Solari 52,"Pubs, Cafes",3.2,6,45.453810,9.157800,POINT (9.1578 45.45381)
17830,"Forze Armate, Milan",Akkademia,Via Carlo Ravizza 1,"Cocktail Bars, Pubs",0.0,0,45.467172,9.152948,POINT (9.15295 45.46717)


In [12]:
# Removing the dupes
Nightlife_nodup = Nightlife.drop_duplicates(subset=['Venue Name','Venue Address','Categories','Average Star Rating',
                                                        'Review Count','geometry'])
# Saving the file
Nightlife_nodup.to_file("Nightlife.geojson", driver="GeoJSON")

Nightlife_nodup

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry
0,"Parco delle Abbazie, Milan",H2O Live Club,Via Mezzomerico 18,Dance Clubs,0.0,0,45.626890,8.625470,POINT (8.62547 45.62689)
1,"Parco delle Abbazie, Milan",049,Piazza Martiri della Libertà 3B,"Lounges, Wine Bars, Cocktail Bars",4.7,3,45.445177,8.616799,POINT (8.6168 45.44518)
2,"Parco delle Abbazie, Milan",Twiggy Club,Via de Cristoforis 5,"Music Venues, Pubs, Cafes",5.0,1,45.821506,8.831431,POINT (8.83143 45.82151)
3,"Parco delle Abbazie, Milan",Circle,Via Stendhal 34,"Lounges, Dance Clubs",3.4,7,45.453430,9.159440,POINT (9.15944 45.45343)
4,"Parco delle Abbazie, Milan",Strix Bar,Via Olgiate Olona 1,Cocktail Bars,4.0,2,45.620991,8.865000,POINT (8.865 45.62099)
...,...,...,...,...,...,...,...,...,...
17777,"Forze Armate, Milan",BAR Piccolo Lord di Comisso,"Via Della Liberazione, 34/A",Bars,0.0,0,45.439727,9.104614,POINT (9.10461 45.43973)
17778,"Forze Armate, Milan",Mazzonna / Antonio,"Piazza Giovanni Xxiii, 7",Bars,0.0,0,45.439020,9.109670,POINT (9.10967 45.43902)
17784,"Forze Armate, Milan",Bonavetti / Andrea,"Via Sebastiano Caboto, 31",Bars,0.0,0,45.443188,9.114465,POINT (9.11447 45.44319)
17790,"Forze Armate, Milan",Cafe' Principe di Massa Antonella,"Via Roma, 108",Bars,0.0,0,45.439091,9.099020,POINT (9.09902 45.43909)


In [14]:
# DEVONO ESSERE LO STESSO TIPO DI COORDINATE
print(Nightlife_nodup.crs)  # Restaurants CRS
print(gdf_combined.crs)  # Neighborhood polygons CRS

EPSG:4326
EPSG:4326


# Join Spaziale

In [16]:
# Perform the spatial join
joined = sjoin(Nightlife_nodup, gdf_combined, how="left", predicate="within")

# Check the result
joined

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry,index_right,Neighborhood
0,"Parco delle Abbazie, Milan",H2O Live Club,Via Mezzomerico 18,Dance Clubs,0.0,0,45.626890,8.625470,POINT (8.62547 45.62689),NaN,NaN
1,"Parco delle Abbazie, Milan",049,Piazza Martiri della Libertà 3B,"Lounges, Wine Bars, Cocktail Bars",4.7,3,45.445177,8.616799,POINT (8.6168 45.44518),NaN,NaN
2,"Parco delle Abbazie, Milan",Twiggy Club,Via de Cristoforis 5,"Music Venues, Pubs, Cafes",5.0,1,45.821506,8.831431,POINT (8.83143 45.82151),NaN,NaN
3,"Parco delle Abbazie, Milan",Circle,Via Stendhal 34,"Lounges, Dance Clubs",3.4,7,45.453430,9.159440,POINT (9.15944 45.45343),74.0,Tortona
4,"Parco delle Abbazie, Milan",Strix Bar,Via Olgiate Olona 1,Cocktail Bars,4.0,2,45.620991,8.865000,POINT (8.865 45.62099),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17777,"Forze Armate, Milan",BAR Piccolo Lord di Comisso,"Via Della Liberazione, 34/A",Bars,0.0,0,45.439727,9.104614,POINT (9.10461 45.43973),NaN,NaN
17778,"Forze Armate, Milan",Mazzonna / Antonio,"Piazza Giovanni Xxiii, 7",Bars,0.0,0,45.439020,9.109670,POINT (9.10967 45.43902),NaN,NaN
17784,"Forze Armate, Milan",Bonavetti / Andrea,"Via Sebastiano Caboto, 31",Bars,0.0,0,45.443188,9.114465,POINT (9.11447 45.44319),NaN,NaN
17790,"Forze Armate, Milan",Cafe' Principe di Massa Antonella,"Via Roma, 108",Bars,0.0,0,45.439091,9.099020,POINT (9.09902 45.43909),NaN,NaN


In [18]:
joined[joined['Venue Name']=='Alchimia'] # giusto

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry,index_right,Neighborhood
5831,"Gratosoglio - Ticinello, Milan",Alchimia,Via Francesco Brioschi 17,"Lounges, Steakhouses",3.9,17,45.446007,9.18065,POINT (9.18065 45.44601),73.0,Ticinese


In [20]:
joined.nunique()

Fetch Location           58
Venue Name             2416
Venue Address          2366
Categories              730
Average Star Rating      38
Review Count             79
Latitude               2297
Longitude              2305
geometry               2368
index_right              77
Neighborhood             77
dtype: int64

In [63]:
# Remove rows where the geometry is NaN
PolyNightlife = joined[~joined["Neighborhood"].isna()]

# Reset the index if needed
PolyNightlife = PolyNightlife.reset_index(drop=True)

# Keeping only relevant variables
PolyNightlife = PolyNightlife[["Venue Name", "Venue Address", "Categories", "Average Star Rating",
                           "Review Count", "geometry", "Neighborhood"]]

# Saving the file
PolyNightlife.to_file("PolyNightlife.geojson", driver="GeoJSON")

# Print the cleaned GeoDataFrame
PolyNightlife

,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,geometry,Neighborhood
0,Circle,Via Stendhal 34,"Lounges, Dance Clubs",3.4,7,POINT (9.15944 45.45343),Tortona
1,Alcatraz,Via Valtellina 25,"Music Venues, Dance Clubs",3.8,39,POINT (9.18275 45.49459),Farini
2,Blue Eyes,Via Luigi De Andreis 13,Dance Clubs,3.0,2,POINT (9.23504 45.46169),Corsica
3,Maremilano,Via Fratelli Zoia 10,Nightlife,5.0,1,POINT (9.10812 45.47635),Quarto Cagnino
4,Monkey,Via Napo Torriani 5,Cocktail Bars,4.8,30,POINT (9.20352 45.48085),Centrale
...,...,...,...,...,...,...,...
1830,Martin / Luigi Pietro,"Viale Monte Nero, 71",Bars,0.0,0,POINT (9.20754 45.4605),Guastalla
1831,BAR White,"Via Friuli, 69",Bars,0.0,0,POINT (9.21479 45.45632),XXII Marzo
1832,Siena / Tommaso,"Viale Premuda, 10",Bars,0.0,0,POINT (9.2077 45.46364),XXII Marzo
1833,Caffè Italia,Viale Premuda 20,Bars,3.5,2,POINT (9.20757 45.46479),XXII Marzo
